In [13]:
import hashlib
import itertools
import json
import random
from time import sleep
from typing import List, Dict, Tuple

import requests
from tqdm import tqdm


In [14]:
RAW_DATA_DIR: str = './raw'

In [ ]:
# baseline 1: persona only
# TODO: create directories for baselines
# directory: baseline_persona_only_GPT35
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["neutral"],
    ["troll"],
    ["conservative"],
    ["dystopian"],
    ["devilsadvocate"],
    ["know-it-all"],
    ["mediator"],
    ["educator"],
    ["karen"],
    ["misogynist"],
    ["esoteric"],
    ["hippie-dense"],
    ["supportive-saviour-dense"],
    ["influencer"],
    ["social-media-activist-dense"],
    ["anti"],
    ["hobby-expert-dense"],
    ["narcissist"],
    ["sceptic"],
    ["shy"],
    ["toxic-positive"],
    ["traditionalist"]
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [ ]:
# baseline 2: political views only
# directory: baseline_political_views_only_GPT35
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["radical-left"],
    ["progressive"],
    ["conservative"],
    ["far-right"],
    ["disengaged"]
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [ ]:
# baseline 3: neutral + political views
# directory: baseline_neutral_political_views_GPT35
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["neutral"],
    ["neutral", "radical-left"],
    ["neutral", "progressive"],
    ["neutral", "conservative"],
    ["neutral", "far-right"],
    ["neutral", "disengaged"]
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [ ]:
# baseline 3: troll combinations
# directory: baseline_troll_GPT35
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["troll"],
    ["troll", "language-default"],
    ["teenager", "troll", "language-youth", "radical-left"],
    ["teenager", "troll", "language-youth", "far-right"],
    ["teenager", "troll", "language-youth", "disengaged"],
    ["unemployed", "troll", "language-youth", "radical-left"],
    ["unemployed", "troll", "language-youth", "far-right"],
    ["unemployed", "troll", "language-youth", "disengaged"]   
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [15]:
# safety triggers test: troll and misogynist
# directory: safety_triggers_GPT35
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["teenager", "troll", "language-youth", "radical-left"],
    ["teenager", "troll", "language-youth", "far-right"],
    ["unemployed", "troll", "language-youth", "radical-left"],
    ["unemployed", "troll", "language-youth", "far-right"],
    ["misogynist", "language-default"],
    ["teenager", "misogynist-dense", "language-youth"],
    ["carpenter", "misogynist", "language-old", "far-right"],
    ["hustler", "misogynist-dense", "language-default", "polit-conservative"]
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [ ]:
integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    ["conservative", "polit-conservative"],
    ["pensioner", "conservative", "language-old", "polit-conservative"],
    ["dystopian", "language-default"],
    ["carpenter", "dystopian", "language-default", "polit-conservative"],
    ["male-college-student", "dystopian", "language-youth", "progressive"],
    ["devilsadvocate", "language-default", "far-right"],
    ["male-college-student", "devilsadvocate", "language-youth", "radical-left"],
    ["pensioner", "devilsadvocate", "language-old", "far-right"],
    ["hustler", "know-it-all", "language-default", "far-right"],
    ["male-college-student", "know-it-all", "language-youth", "radical-left"],
    ["know-it-all", "language-default", "progressive"],
    ["retired-professor", "educator", "language-old", "progressive"],
    ["priviledged-woman", "karen", "language-default", "far-right"],
    ["affluent-young-woman", "karen", "language-default", "polit-conserative"],
    ["misogynist-dense", "language-default"],
    ["teenager", "misogynist-dense", "language-youth"],
    ["carpenter", "misogynist-dense", "language-old", "far-right"],
    ["hustler", "misogynist-dense", "language-default", "polit-conservative"],
    ["esoteric", "language-default", "radical-left"],
    ["priviledged woman", "esoeric", "language-default", "far-right"],
    ["hippie", "language-default"],
    ["priviledged-woman", "hippie", "language-default", "radical-left"],
    ["phd-student", "supportive-saviour-dense", "language-default", "progressive"],
    ["phd-student", "supportive-saviour", "language-default", "progressive"],
    ["hustler", "influencer-dense", "language-default", "progressive"],
    ["wealthy-entrepreneur", "influencer", "language-default", "polit-conservative"],
    ["affluent-young-woman", "social-media-activist-dense", "language-default", "radical-left"],
    ["male-college-student", "social-media-activist", "language-youth", "far-right"],
    ["retired-professor", "anti", "language-old", "far-right"],
    ["retired-professor", "anti", "language-old", "radical-left"],
    ["office-worker", "anti", "language-default", "polit-conservative"],
    ["teenager", "anti", "language-youth", "radical-left"],
    ["teenager", "anti", "language-youth", "far-right"],
    ["pensioner", "toxic-positive", "language-old", "disengaged"],
    ["phd-student", "toxic-positive", "language-default"],
    ##["affluent-young-woman", "traditionalist", "language-default", "progressive"],
    ["wealthy-entrepreneur", "traditionalist", "language-default", "polit-conservative"],
    ##
    ["retired-professor", "traditionalist", "language-old", "polit-conservative"],
    ["retired-professor", "traditionalist", "language-old", "far-right"],
    ["unemployed", "traditionalist", "language-default", "polit-conservative"],
    ["single-parent", "sceptic", "language-default", "polit-conservative"],
    ["single-parent", "sceptic", "language-old", "far-right"],
    ["environmental-activist", "sceptic", "language-default", "progressive"],
    ["single-parent", "sceptic", "language-youth", "progressive"],
    ["struggling-artist", "shy", "language-default", "disengaged"],
    ["struggling-artist", "shy", "language-old", "polit-conservative"],
    ["teenager", "shy", "language-youth", "radical-left"],
    ["phd-student", "shy", "language-default", "disengaged"],
    ["priviledged-woman", "narcissist", "language-default", "far-right"],
    ["teenager", "narcissist", "language-youth", "radical-left"]
]
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [77]:
# experiment setup

integrations: List[dict] = [
    #{
    #    "provider": "huggingFace",
    #    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    #},
    {
        "provider": "OpenAI",
        "model": "gpt-3.5-turbo"
    }
]

personas: List[List[str]] = [
    #["devilsadvocate", "far-right"],
    ["male-college-student", "language-youth", "devilsadvocate", "radical-left"],
    #["pensioner", "devilsadvocate", "far-right"],
    ["know-it-all", "progressive"],
    #["hustler", "know-it-all", "far-right"],
    ["male-college-student", "language-youth", "know-it-all", "radical-left"],
    #["phd-student", "mediator", "progressive"],
    ["affluent-young-woman", "mediator", "polit-conservative"],
    ["privileged-woman", "karen", "far-right"],
    #["affluent-young-woman", "karen", "polit-conservative"],
    #["teenager", "language-youth", "misogynist"],
    ["carpenter", "misogynist", "far-right"],
    ["privileged-woman", "esoteric", "far-right"],
    ["privileged-woman", "hippie", "radical-left"],
    #["phd-student", "supportive-saviour", "progressive"],
    ["hustler", "influencer", "progressive"],
    #["wealthy-entrepreneur", "influencer", "polit-conservative"],
    ["affluent-young-woman", "social-media-activist", "radical-left"],
    #["male-college-student", "language-youth", "social-media-activist", "far-right"]
]
#languages: List[str] = ["Dutch", "English", "German"]
#platforms: List[str] = ["Twitter", "Reddit"] 
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [16]:
# number of iterations
x: int = 1
# hidden parameters (randomly chosen)
length: List[str] = ["50", "100", "200", "280"] 

In [17]:
len(threads)

9

In [18]:
configurations: List[Tuple] = list(
    itertools.product(*[integrations, personas, threads])
)
random.shuffle(configurations)
len(configurations)

72

In [19]:
configurations[0]

({'provider': 'OpenAI', 'model': 'gpt-3.5-turbo'},
 ['hustler', 'misogynist-dense', 'language-default', 'polit-conservative'],
 {'persona': 'conservative',
  'model': 'gpt-3.5-turbo',
  'topic': 'Economy',
  'platform': 'Twitter',
  'language': 'English',
  'text': 'Inflation running rampant, eroding the purchasing power of hardworking Americans and threatening our economic stability - time to reign in government spending and uphold the principles of fiscal responsibility for a prosperous future! 💪🏼💵📉',
  'annotation': {'topic': None, 'persona': None, 'authenticity': None},
  'id': '1c961e904e1c7c4de32c447c3df3a48797bb4a7a7674514b'})

In [21]:
for _ in range(x):
    for integration, persona, thread in tqdm(configurations):

        payload: dict = {
            "personas": persona,
            "integration": integration,
            "thread": {'posts': [{'author': thread['persona'], 'message': thread['text']}]},
            "length": random.choice(length)
        }

        response = requests.post('http://127.0.0.1:8000/reply/', json=payload)

        if response.status_code == 500 or response.status_code == 502:
            print("500/502: Connection Error, too many request, try again later.")
            break

        try:
            data: dict = response.json()

        except Exception as e:
            print(e, ':', response)
            break
        
        sample: dict = {
            "persona": data['persona']['id'],
            "model": integration["model"],
            "original_author": thread['persona'],
            "threads": thread['text'],
            "topic": thread["topic"],
            "text": data['response'],
            "payload": data,
            "annotation": {
                "politics": None,
                "persona": None,
                "tone": None,
                "authenticity": None
            }
        }
        print(data['persona'])
        print(data['response'])
        sample['id'] = hashlib.shake_256(str.encode(json.dumps(sample))).hexdigest(24)
        open(f'{RAW_DATA_DIR}/{sample["id"]}.json', 'w').write(json.dumps(sample, indent=4))
        sleep(1)


  0%|          | 0/72 [00:00<?, ?it/s]

@conservative Agreed! Government needs to stop spending like there's no tomorrow.


  1%|▏         | 1/72 [00:03<03:37,  3.06s/it]

@liberal ur agenda won't create jobs, only take from those working hard. Stop virtue signaling.


  3%|▎         | 2/72 [00:06<03:33,  3.05s/it]

...

 97%|█████████▋| 70/72 [22:29<00:37, 18.63s/it]

@conservative hahaha ur so right!! 😂👏 'radical climate agenda' sounds hella crazy, like what's next? mandatory veggie burgers and electronic car races? 🤔 I guess freedom is just way too un-American these days! 🇺🇸


 99%|█████████▊| 71/72 [22:54<00:20, 20.53s/it]

it's not about politics, it's about protecting innocent lives. abortion is murder, plain and simple. no exceptions needed.


100%|██████████| 72/72 [23:17<00:00, 19.41s/it]
